# Sysbench - CPU Workloads: Firecracker microVM

In [21]:
import re
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. Parse Sysbench Results

In [22]:
df = pd.DataFrame(columns=['Number of vCPUs',
                           'Number of Threads',
                           'Max Prime',
                           'Events per second',
                           'Total Time',
                           'Total Number of Events',
                           'Min Latency (ms)',
                           'Avg Latency (ms)',
                           'Max Latency (ms)',
                           '95th Percentile Latency (ms)',
                           'Total Execution Time across Threads (s)',
                           'Avg Number of Events per Thread',
                           'Avg Execution Time per Thread (s)'])

In [23]:
df

,Number of vCPUs,Number of Threads,Max Prime,Events per second,Total Time,Total Number of Events,Min Latency (ms),Avg Latency (ms),Max Latency (ms),95th Percentile Latency (ms),Total Execution Time across Threads (s),Avg Number of Events per Thread,Avg Execution Time per Thread (s)


In [24]:
def readFromOutputs(filepath, num_vCPUs):
    for outputFile in os.listdir(filepath):
        linenum = 0
        full_filepath = os.path.join(filepath, outputFile)
        if outputFile == '.gitkeep':
            continue
        with open(full_filepath, 'r') as f:
            testResult = [num_vCPUs]
            for line in f:
                if ':' in line:
                    result = line.rsplit(':', 1)[1].strip()
                    if 's' in result:
                        testResult.append(float(result[:-1]))
                    elif '/' in result:
                        testResult.append(float(result.split('/')[0]))
                    elif result != '':
                        testResult.append(float(result))
            df.loc[len(df)] = testResult

readFromOutputs('./microVM4vCPUOutputs', 4)
readFromOutputs('./microVM30vCPUOutputs', 30)

In [25]:
df = df.astype({'Number of vCPUs': int, 'Number of Threads': int, 'Max Prime': int})\
       .sort_values(by=['Number of vCPUs', 'Number of Threads', 'Max Prime']).reset_index(drop=True)

df['Total Execution Time across Threads (s)'] = df['Total Execution Time across Threads (s)'] / 1000

In [26]:
df.head()

,Number of vCPUs,Number of Threads,Max Prime,Events per second,Total Time,Total Number of Events,Min Latency (ms),Avg Latency (ms),Max Latency (ms),95th Percentile Latency (ms),Total Execution Time across Threads (s),Avg Number of Events per Thread,Avg Execution Time per Thread (s)
0,4,1,10000,1313.00,30.0011,39398.0,0.73,0.76,6.93,0.78,29.96913,39398.0,29.9691
1,4,1,50000,140.04,30.0008,4202.0,7.01,7.14,15.98,7.17,29.99060,4202.0,29.9906
2,4,1,100000,53.02,30.0021,1591.0,18.62,18.85,26.85,18.95,29.99700,1591.0,29.9970
3,4,1,500000,5.45,30.0770,164.0,181.70,183.38,201.22,189.93,30.07458,164.0,30.0746
4,4,1,1000000,2.04,30.4351,62.0,486.69,490.84,504.31,502.20,30.43227,62.0,30.4323


In [27]:
df.tail()

,Number of vCPUs,Number of Threads,Max Prime,Events per second,Total Time,Total Number of Events,Min Latency (ms),Avg Latency (ms),Max Latency (ms),95th Percentile Latency (ms),Total Execution Time across Threads (s),Avg Number of Events per Thread,Avg Execution Time per Thread (s)
245,30,40,10000,22044.93,30.0158,661729.0,0.73,1.81,89.37,1.67,1198.90727,16543.225,29.9727
246,30,40,50000,2284.53,30.0242,68597.0,7.03,17.49,151.04,29.72,1199.84439,1714.925,29.9961
247,30,40,100000,858.52,30.0489,25801.0,18.72,46.52,226.99,84.47,1200.19805,645.025,30.0050
248,30,40,500000,87.59,30.2969,2654.0,184.51,454.40,1041.64,733.00,1205.97868,66.350,30.1495
249,30,40,1000000,32.40,30.8620,1000.0,493.09,1216.20,2155.69,1938.16,1216.19558,25.000,30.4049


In [28]:
df.to_csv('sysbench_microVM.csv')

## 2. Visualization

In [29]:
dff_4vCPU_100k = df.loc[(df['Number of vCPUs'] == 4) & (df['Max Prime'] == 100000)]
dff_4vCPU_1m = df.loc[(df['Number of vCPUs'] == 4) & (df['Max Prime'] == 1000000)]

plot1 = go.Scatter(x=dff_4vCPU_100k['Number of Threads'],
                   y=dff_4vCPU_100k['Events per second'],
                   mode='markers+lines',
                   name='Max Prime = 100,000')
 
plot2 = go.Scatter(x=dff_4vCPU_1m['Number of Threads'],
                   y=dff_4vCPU_1m['Events per second'],
                   mode='markers+lines',
                   name='Max Prime = 1,000,000')
 

xaxis_title="Number of Threads"
yaxis_title="Events per second"

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(plot1, 1, 1)
fig.append_trace(plot2, 2, 1)
fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
fig.update_yaxes(title_text=yaxis_title, row=2, col=1)
fig.update_xaxes(title_text=xaxis_title, row=1, col=1)
fig.update_xaxes(title_text=xaxis_title, row=2, col=1)
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="green")
fig['layout'].update(height=600, width=1000,
                     title='CPU Speed (Eps) on a bare-metal server with 4 logical cores')
fig.show()

In [17]:
plot1 = go.Scatter(x=dff_4vCPU_100k['Number of Threads'],
                   y=dff_4vCPU_100k['Avg Latency (ms)'],
                   mode='markers+lines',
                   name='Max Prime = 100,000')
 
plot2 = go.Scatter(x=dff_4vCPU_1m['Number of Threads'],
                   y=dff_4vCPU_1m['Avg Latency (ms)'],
                   mode='markers+lines',
                   name='Max Prime = 1,000,000')
 

xaxis_title="Number of Threads"
yaxis_title="Avg Latency (ms)"

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(plot1, 1, 1)
fig.append_trace(plot2, 2, 1)
fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
fig.update_yaxes(title_text=yaxis_title, row=2, col=1)
fig.update_xaxes(title_text=xaxis_title, row=1, col=1)
fig.update_xaxes(title_text=xaxis_title, row=2, col=1)
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="green")
fig['layout'].update(height=600, width=1000,
                     title='Average Latency for an Event by a Thread on a microVM with 4 logical cores')
fig.show()

In [33]:
dff_4vCPU_1m = df.loc[(df['Number of vCPUs'] == 4) & (df['Max Prime'] == 1000000)]
dff_30vCPU_1m = df.loc[(df['Number of vCPUs'] == 30) & (df['Max Prime'] == 1000000)]

plot1 = go.Scatter(x=dff_4vCPU_1m['Number of Threads'],
                   y=dff_4vCPU_1m['Avg Latency (ms)'],
                   mode='markers+lines',
                   name='4vCPUs')
 
plot2 = go.Scatter(x=dff_30vCPU_1m['Number of Threads'],
                   y=dff_30vCPU_1m['Avg Latency (ms)'],
                   mode='markers+lines',
                   name='30 vCPUs')
 

xaxis_title="Number of Threads"
yaxis_title="Avg Latency (ms)"

fig = make_subplots(rows=2, cols=1, shared_xaxes=False)
fig.append_trace(plot1, 1, 1)
fig.append_trace(plot2, 2, 1)
fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
fig.update_yaxes(title_text=yaxis_title, row=2, col=1)
fig.update_xaxes(title_text=xaxis_title, row=1, col=1)
fig.update_xaxes(title_text=xaxis_title, row=2, col=1)
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="green")
fig['layout'].update(height=600, width=1000,
                     title='Average Latency for an Event by a Thread on a microVM (Max Prime = 1mil)')
fig.show()

In [45]:
dff = df.loc[((df['Number of vCPUs'] == 4) | (df['Number of vCPUs'] == 30)) & (df['Max Prime'] == 1000000)]

fig = px.line(dff, x='Number of Threads', y='Avg Latency (ms)', color='Number of vCPUs', markers=True)
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="blue")
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="blue")
fig.add_vline(x=15, line_width=1, line_dash="dot", line_color="red")
fig.add_vline(x=30, line_width=1, line_dash="dot", line_color="red")
fig['layout'].update(title='Average Latency for an Event by a Thread on a microVM (Max Prime = 1mil)')
fig.show()

In [46]:
fig = px.line(dff, x='Number of Threads', y='Events per second', color='Number of vCPUs', markers=True)
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="blue")
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="blue")
fig.add_vline(x=15, line_width=1, line_dash="dot", line_color="red")
fig.add_vline(x=30, line_width=1, line_dash="dot", line_color="red")
fig['layout'].update(title='CPU Speed (Eps) on a microVM (Max Prime = 1mil)')
fig.show()

In [47]:
fig = px.line(dff, x='Number of Threads', y='Total Number of Events', color='Number of vCPUs', markers=True)
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="blue")
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="blue")
fig.add_vline(x=15, line_width=1, line_dash="dot", line_color="red")
fig.add_vline(x=30, line_width=1, line_dash="dot", line_color="red")
fig['layout'].update(title='Total Number of Events on a microVM (Max Prime = 1mil)')
fig.show()